In [53]:
import sys
import os
os.environ['PROJ_DATA'] = "/pscratch/sd/p/plutzner/proj_data"
import xarray as xr
import torch
import torchinfo
import random
import numpy as np
import importlib as imp
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import cartopy.crs as ccrs
import json
import pickle
import gzip
#import matplotlib.colors as mcolorsxx

%load_ext autoreload
%autoreload 2
import utils
import utils.filemethods as filemethods
import databuilder.data_loader as data_loader
import model.loss as module_loss
import model.metric as module_metric
from databuilder.data_generator import multi_input_data_organizer
import databuilder.data_loader as data_loader
from trainer.trainer import Trainer
from model.build_model import TorchModel
from utils import utils
# import databuilder.nino_indices as nino_indices # CAUSES CELL TO HANG

print(f"python version = {sys.version}")
print(f"numpy version = {np.__version__}")
print(f"xarray version = {xr.__version__}")
print(f"pytorch version = {torch.__version__}")

# https://github.com/victoresque/pytorch-template/tree/master

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
python version = 3.10.10 | packaged by conda-forge | (main, Mar 24 2023, 20:08:06) [GCC 11.3.0]
numpy version = 1.26.4
xarray version = 2024.5.0
pytorch version = 2.0.0.post104


In [54]:
config = utils.get_config("exp002")
seed = config["seed_list"][0]

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True

Process + Pickle Inputs and Targets

In [90]:
# s_dict_train, s_dict_val, s_dict_test = multi_input_data_organizer(config)

# s_dict_train["x"][:124]

# s_dict_savename1 = '/pscratch/sd/p/plutzner/E3SM/bigdata/presaved/exp002_train.pkl'
# with gzip.open(s_dict_savename1, "wb") as fp:
#     pickle.dump(s_dict_train, fp)

# s_dict_savename2 = '/pscratch/sd/p/plutzner/E3SM/bigdata/presaved/exp002_val.pkl'
# with gzip.open(s_dict_savename2, "wb") as fp:
#     pickle.dump(s_dict_val, fp)

# s_dict_savename3 = '/pscratch/sd/p/plutzner/E3SM/bigdata/presaved/exp002_test.pkl'
# with gzip.open(s_dict_savename3, "wb") as fp:
#     pickle.dump(s_dict_test, fp)

Opening MJO PCs
Opening Nino34 Data
/pscratch/sd/p/plutzner/E3SM/bigdata/E3SMv2data/member0101/member0101.Nino34.daily.int.nc
Opening exp001 to extract target data for TRAINING
Opening exp001 to extract target data for VALIDATION
Opening exp001 to extract target data for TESTING
Combining Input and target data
(60211, 3, 3)


Retrieve Data: 


In [55]:
# Setup the Data
trainset = data_loader.CustomData(config["data_loader"]["data_dir"] + "exp002_train.pkl")
valset = data_loader.CustomData(config["data_loader"]["data_dir"] + "exp002_val.pkl")

train_loader = torch.utils.data.DataLoader(
    trainset,
    batch_size=config["data_loader"]["batch_size"],
    shuffle=True,
    drop_last=False,
)

val_loader = torch.utils.data.DataLoader(
    valset,
    batch_size=config["data_loader"]["batch_size"],
    shuffle=False,
    drop_last=False,
)

X1 shape: (60090,)
Input unit shape: (60090,)
X1 shape: (60090,)
Input unit shape: (60090,)


Model Setup

In [56]:
# Setup the Model
model = TorchModel(
    config=config["arch"],
    target_mean=trainset.target.mean(axis=0),
    target_std=trainset.target.std(axis=0),
)
model.freeze_layers(freeze_id="tau")
optimizer = getattr(torch.optim, config["optimizer"]["type"])(
    model.parameters(), **config["optimizer"]["args"]
)
criterion = getattr(module_loss, config["criterion"])()
metric_funcs = [getattr(module_metric, met) for met in config["metrics"]]

# Build the trainer
device = utils.prepare_device(config["device"])
trainer = Trainer(
    model,
    criterion,
    metric_funcs,
    optimizer,
    max_epochs=config["trainer"]["max_epochs"],
    data_loader=train_loader,
    validation_data_loader=val_loader,
    device=device,
    config=config,
)


In [57]:
# Visualize the model
torchinfo.summary(
    model,
    [   trainset.x1[: config["data_loader"]["batch_size"]].shape,
        trainset.x2[: config["data_loader"]["batch_size"]].shape,
        trainset.x3[: config["data_loader"]["batch_size"]].shape
    ],
    verbose=1,
    col_names=("input_size", "output_size", "num_params"),
)

# Train the Model
model.to(device)
trainer.fit()

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
TorchModel                               [32]                      [32, 4]                   --
├─Linear: 1-1                            [32, 3]                   [32, 3]                   12
├─Linear: 1-2                            [32, 3]                   [32, 3]                   12
├─Flatten: 1-3                           [32, 3]                   [32, 3]                   --
├─Sequential: 1-4                        [32, 3]                   [32, 3]                   --
│    └─Sequential: 2-1                   [32, 3]                   [32, 3]                   --
│    │    └─Linear: 3-1                  [32, 3]                   [32, 3]                   12
│    │    └─ReLU: 3-2                    [32, 3]                   [32, 3]                   --
│    └─Sequential: 2-2                   [32, 3]                   [32, 3]                   --
│    │    └─Linear: 3-3            

TypeError: unsupported format string passed to list.__format__